In [71]:
from typing import Callable, Tuple
import pandas as pd
import numpy as np

In [72]:
func: Callable[[float], float] = lambda x: 3 * x**2 + (12 / x**3) - 5
#a = 0.5
#b = 2.5
#eps = 0.05
a = float(input("Введите начало промежутка"))
b = float(input("Введите конец промежутка"))
eps = float(input("Введите погрешность"))

## Задание 1

In [73]:
def scan(func: Callable[[float], float], a: float, b: float, eps: float = 0.05) -> float:
    steps = int((b - a) / eps)
    x_values = np.linspace(a, b, steps) 
    values = func(x_values)

    return np.min(values) 

scan(func, a, b, eps)

5.239303775531791

In [74]:
def scan(func: Callable[[float], float], a: float, b: float, eps: float = 0.05) -> Tuple[float, float]:
    steps = int((b - a) / eps)
    x_values = np.linspace(a, b, steps) 
    values = func(x_values)

    return np.min(values), len(x_values)

scan(func, a, b, eps)

(5.239303775531791, 40)

In [ ]:
count_segments = float(input("Введите кол-во отрезков"))
#count_segments = 4

In [75]:
def scan_variable_step(func: Callable[[float], float], a: float, b: float, eps: float = 0.05,
                       count_segment: int = 4) -> float:
    
    while((b-a)/2 > eps):
        intervals = np.linspace(a, b, count_segment+1)  # Генерация n+1 точек на отрезке
        midpoints = (intervals[:-1] + intervals[1:]) / 2  # Вычисление середины каждого интервала
        values = func(midpoints)
        min_index = np.argmin(values)
        a, b = intervals[min_index], intervals[min_index + 1]
        
    return func((b + a)/2)

scan_variable_step(func, a, b, eps, count_segments)

5.239613702947885

In [76]:
def scan_variable_step(func: Callable[[float], float], a: float, b: float, eps: float = 0.05,
                       count_segment: int = 4) -> Tuple[float, float]:
    
    count = 0
    while((b-a)/2 > eps):
        intervals = np.linspace(a, b, count_segment+1)  # Генерация n+1 точек на отрезке
        midpoints = (intervals[:-1] + intervals[1:]) / 2  # Вычисление середины каждого интервала
        values = func(midpoints)
        count += len(midpoints)
        min_index = np.argmin(values)
        a, b = intervals[min_index], intervals[min_index + 1]
        
    return func((b + a)/2), count

value, count = scan_variable_step(func, a, b, eps, count_segment=4)
print(value, count)

5.239613702947885 12


## Задание 2

In [77]:
def half_division(func: Callable[[float], float], a: float, b: float, eps: float = 0.05) -> float:
    while (b - a) / 2 > eps:
        x = (b + a) / 2  # Вычисление середины интервала
        value_func_left, value_func_right = func(x - eps/2), func(x+eps/2)
        if value_func_left < value_func_right:
            b = x
        else:
            a = x
            
    return func((b + a) / 2)

half_division(func, a, b, eps)
    

5.247744759516461

In [78]:


def half_division(func: Callable[[float], float], a: float, b: float,
                  eps: float = 0.05) -> Tuple[pd.DataFrame, float]:
    data = []  # Для хранения данных на каждой итерации
    count = 0
    while (b - a) / 2 > eps:
        x = (b + a) / 2  # Вычисление середины интервала
        x_left = x - eps / 2
        x_right = x + eps / 2
        value_func_left = func(x_left)
        value_func_right = func(x_right)
        count +=2
        if value_func_left < value_func_right:
            min_side = 'right'
            b = x
        else:
            min_side = 'left'
            a = x
        
        # Добавление данных об этой итерации в список
        data.append([a, b, x_left, x, x_right, value_func_left, value_func_right, count, min_side])
    
    # Создание DataFrame из списка данных
    columns = ['a', 'b', 'c-E/2', 'c', 'c+E/2', 'func(c-E/2)', 'func(c+E/2)', 'count', 'Min side']
    df = pd.DataFrame(data, columns=columns)
    return df, func((b+a)/2)

df, value = half_division(func, a, b, eps)
print(df)
print(value)

       a       b   c-E/2       c   c+E/2  func(c-E/2)  func(c+E/2)  count  \
0  0.500  1.5000  1.4750  1.5000  1.5250     5.266303     5.360418      2   
1  1.000  1.5000  0.9750  1.0000  1.0250    10.798823     9.295068      4   
2  1.250  1.5000  1.2250  1.2500  1.2750     6.029767     5.666503      6   
3  1.375  1.5000  1.3500  1.3750  1.4000     5.344805     5.253178      8   
4  1.375  1.4375  1.4125  1.4375  1.4625     5.243569     5.252851     10   

  Min side  
0    right  
1     left  
2     left  
3     left  
4    right  
5.247744759516461


## Задание 6

In [79]:
def golden_section_search(func: Callable[[float], float], a: float, b: float,
                  eps: float = 0.05) -> float:
    
    phi = (1 + np.sqrt(5)) / 2
    resphi = 2 - phi
    x1 = a + resphi * (b - a)
    x2 = b - resphi * (b - a)
    f1 = func(x1)
    f2 = func(x2)
    
    while (b - a)/2 > eps:
        if f1 < f2:
            b = x2
            x2 = x1
            f2 = f1
            x1 = a + resphi * (b - a)
            f1 = func(x1)
        else:
            a = x1
            x1 = x2
            f1 = f2
            x2 = b - resphi * (b - a)
            f2 = func(x2)
    
    return func((b+a)/2)
        
    
golden_section_search(func, a, b, eps)

5.245200442713884

In [80]:
from typing import Callable
import pandas as pd

def golden_section_search(func: Callable[[float], float], a: float, b: float,
                          eps: float = 0.05)-> Tuple[pd.DataFrame, float]:
    count = 0
    phi = (1 + np.sqrt(5)) / 2
    resphi = 2 - phi
    x1 = a + resphi * (b - a)
    x2 = b - resphi * (b - a)
    f1 = func(x1)
    f2 = func(x2)
    count += 2
    
    data = []  # Для хранения данных на каждой итерации
    while (b - a) / 2 > eps:
        row = [a, b, x1, func(x1), x2, count, func(x2)]  # Данные об этой итерации
        data.append(row)
        
        if f1 < f2:
            b = x2
            x2 = x1
            f2 = f1
            x1 = a + resphi * (b - a)
            f1 = func(x1)
            count += 1
        else:
            a = x1
            x1 = x2
            f1 = f2
            x2 = b - resphi * (b - a)
            f2 = func(x2)
            count += 1
    
    columns = ['a', 'b', 'x1', 'func(x1)', 'x2', 'count', 'func(x2)']
    df = pd.DataFrame(data, columns=columns)
    return df, func((a+b)/2)

df, value = golden_section_search(func, a, b, eps)
print(df)
print(value)

          a         b        x1   func(x1)        x2  count  func(x2)
0  0.500000  2.500000  1.263932   5.735632  1.736068      2  6.335203
1  0.500000  1.736068  0.972136  10.896861  1.263932      3  5.735632
2  0.972136  1.736068  1.263932   5.735632  1.444272      4  5.240985
3  1.263932  1.736068  1.444272   5.240985  1.555728      5  5.447856
4  1.263932  1.555728  1.375388   5.287249  1.444272      6  5.240985
5  1.375388  1.555728  1.444272   5.240985  1.486844      7  5.282891
6  1.375388  1.486844  1.417961   5.240932  1.444272      8  5.240985
5.245200442713884


## Задание 10

In [81]:
def quadratic_interpolation(func: Callable[[float], float], A: float, B: float, eps: float = 0.05) -> float:

    x0, y0 = (B+A)/2, func((B+A)/2)
    while True:
        x1, y1 = x0-eps/2, func(x0-eps/2)
        x2, y2 = x0+eps/2, func(x0+eps/2)
        
            # Решение системы уравнений
        A = np.array([[x1**2, x1, 1],
                    [x2**2, x2, 1],
                    [x0**2, x0, 1]])
        B = np.array([y1, y2, y0])
        coeffs = np.linalg.solve(A, B)

        # Коэффициенты a, b, c
        a, b, c = coeffs
        x_new = -b/(2*a)
        if (abs(x_new - x0) < eps):
            break
        else:
            x0, y0 = x_new, func(x_new)
    
    return func(x_new)

quadratic_interpolation(func, a, b, eps)


5.23836337537727

In [82]:
def quadratic_interpolation(func: Callable[[float], float], A: float, B: float,
                            eps: float = 0.05) -> Tuple[pd.DataFrame, float]:
    
    count = 0
    data = []
    x0, y0 = (B + A) / 2, func((B + A) / 2)
    count += 1
    
    while True:
        x1, x2 = x0 - eps / 2, x0 + eps / 2
        y1, y2 = func(x1), func(x2)
        count+=2
        
        # Решение системы уравнений
        A_matrix = [[x1**2, x1, 1],
                    [x2**2, x2, 1],
                    [x0**2, x0, 1]]
        B_matrix = [y1, y2, y0]
        coeffs = np.linalg.solve(A_matrix, B_matrix)
        a, b, c = coeffs
        

        
        # Вычисление новой x координаты минимума
        x_new = -b / (2 * a)
        
        # Сохранение промежуточных вычислений
        row = [x0, x1, x2, a, b, c, x_new, count, func(x0)]  # Данные об этой итерации
        data.append(row)
        
        if abs(x_new - x0) < eps:
            break
        else:
            x0, y0 = x_new, func(x_new)
            count+=1
        
    df = pd.DataFrame(data, columns=['x0', 'x1', 'x2', 'a', 'b', 'c', 'x_new', 'count', 'func(x0)'])
    return df, func(x_new)

df, value = quadratic_interpolation(func, a, b, eps)
print(df)
print(value)

         x0        x1        x2          a          b          c     x_new  \
0  1.500000  1.475000  1.525000  12.488069 -35.581907  30.580260  1.424636   
1  1.424636  1.399636  1.449636  15.278555 -43.733426  36.534012  1.431203   

   count  func(x0)  
0      3  5.305556  
1      6  5.238968  
5.23836337537727
